In [189]:
import os
from itertools import permutations
from itertools import product
from typing import Callable, Dict, List, Optional, Tuple, Union

import os
os.chdir('../')  # 更改到上一级目录
import numpy as np
import pandas as pd
import torch
from argoverse.map_representation.map_api import ArgoverseMap
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from tqdm import tqdm
from utils import TemporalData

In [190]:
def get_lanes( raw_path: str,
              raw_path_obs: str,
              frame_to_get: int
              )-> Dict:
    

    
    obs_df = pd.read_csv(raw_path_obs)  
    timestamps = list(np.sort(obs_df['frame_idx'].unique()))
    timestamps = timestamps[frame_to_get-19:frame_to_get+31]##
    if(frame_to_get<19):
        print('Not Enought History')
        return
    
    print('from to ',timestamps[0],timestamps[-1])
    obs_df = obs_df[obs_df['frame_idx'].isin(timestamps)]
    historical_timestamps = timestamps[: 20]
    historical_obs_df = obs_df[obs_df['frame_idx'].isin(historical_timestamps)] 
    # history time timestamp -> 筛选ID种类 
    actor_ids = list(historical_obs_df['track_id'].unique())#历史内所有目标的序列ID 是一个包含所有目标 ID 的列表。
    #回到全frame 用历史ID种类  筛选整个frame 保证  frame里面都是历史出现过的ID 包含了历史和未来
    obs_df = obs_df[obs_df['track_id'].isin(actor_ids)] #保留历史出现过的ID
    
    actor_num = len(actor_ids)# 历史中有的ID
    print('actors ids :',actor_ids)

    
    av_df = obs_df[obs_df['object_type'] == 1].iloc #自动驾驶车的 frame
    av_index = actor_ids.index(av_df[0]['track_id'])#av  在 actor_ids 列表中的索引位置。
    agent_index = 2 # null

    # ready to make the scene centered at AV
    origin = torch.tensor([av_df[19]['rel_x'], av_df[19]['rel_y']], dtype=torch.float)
    print('origin:', origin)
    # origin = torch.tensor([av_df[19]['rel_x'], av_df[19]['rel_y']], dtype=torch.float)
    av_heading_vector = origin - torch.tensor([av_df[18]['rel_x'], av_df[18]['rel_y']], dtype=torch.float)#
   
    theta = torch.atan2(av_heading_vector[1], av_heading_vector[0])
    print('av theta in boost at 19 stamps ',theta)
    rotate_mat = torch.tensor([[torch.cos(theta), -torch.sin(theta)],
                               [torch.sin(theta), torch.cos(theta)]])#实际上不需要  local region的 agent centric
    
     # initialization
    x = torch.zeros(actor_num, 50, 2, dtype=torch.float)
    edge_index = torch.LongTensor(list(permutations(range(actor_num), 2))).t().contiguous()#(2, num_edges)且连续的有向边
    padding_mask = torch.ones(actor_num, 50, dtype=torch.bool)
    bos_mask = torch.zeros(actor_num, 20, dtype=torch.bool)
    rotate_angles = torch.zeros(actor_num, dtype=torch.float)
    complete_samples=[]
    for actor_id, actor_df in obs_df.groupby('track_id'):# 以ID为中心出发更新 将 DataFrame 按照 TRACK_ID 列进行分组，每个分组包含一个 actor_id 和相应的数据actor_df
        node_idx = actor_ids.index(actor_id)
        # print('actor_id : ',actor_id)
        # print('node_idx : ',node_idx)
        node_steps = [timestamps.index(timestamp) for timestamp in actor_df['frame_idx']]# 在最原始文件中的 时间戳的位置
        
        # print('node_steps : ',node_steps)
        padding_mask[node_idx, node_steps] = False
        if padding_mask[node_idx, 19]:  # make no predictions for actors that are unseen at the current time step
            padding_mask[node_idx, 20:] = True #如果在时间步 19 的掩码仍然是 True，则表示该节点在时间步 19 之后的所有时间步都没有数据，因此将这些时间步的掩码设置为 True
        xy = torch.from_numpy(np.stack([actor_df['rel_x'].values, actor_df['rel_y'].values], axis=-1)).float() #当前的actor的轨迹点序列 沿着时间向下延展
        x[node_idx, node_steps] = torch.matmul(xy - origin , rotate_mat)# 
        
        
        node_historical_steps = list(filter(lambda node_step: node_step < 20, node_steps))
        # print('node_historical_steps : ',node_historical_steps)
        if len(node_historical_steps)==20:
            print('full node_historical_steps stamps:', node_idx)
        if len(node_steps)==50:
            print('full _steps stamps:', node_idx)
            complete_samples.append(node_idx)
        if len(node_historical_steps) > 1:  # calculate the heading of the actor (approximately)
            heading_vector = x[node_idx, node_historical_steps[-1]] - x[node_idx, node_historical_steps[-2]] # 朝向
            rotate_angles[node_idx] = torch.atan2(heading_vector[1], heading_vector[0])
        else:  # make no predictions for the actor if the number of valid time steps is less than 2
            padding_mask[node_idx, 20:] = True
    
    # bos_mask is True if time step t is valid and time step t-1 is invalid
    bos_mask[:, 0] = ~padding_mask[:, 0]# break/begining-of-Sequence Mask
    bos_mask[:, 1: 20] = padding_mask[:, : 19] & ~padding_mask[:, 1: 20] # 某个时间t试试是有效的并且之前的时间是无效的 标记那些在时间步 t 有数据而时间步 t-1 需要填充的时间步
    positions = x.clone()#差分前， 保留原始轨迹位置，以AV为中心

     #使用偏差
    x[:, 20:] = torch.where((padding_mask[:, 19].unsqueeze(-1) | padding_mask[:, 20:]).unsqueeze(-1),
                            torch.zeros(actor_num, 30, 2),
                            x[:, 20:] - x[:, 19].unsqueeze(-2))# 未来时刻-当前时刻的偏差 torch.where(condition, x_if_true, x_if_false)
    x[:, 1: 20] = torch.where((padding_mask[:, : 19] | padding_mask[:, 1: 20]).unsqueeze(-1),
                              torch.zeros(actor_num, 19, 2),
                              x[:, 1: 20] - x[:, : 19])#1-19状态   0-18状态
    x[:, 0] = torch.zeros(actor_num, 2)#将第一个时间步的值设置为零

    # get lane features at the current time step
    df_19 = obs_df[obs_df['frame_idx'] == timestamps[19]]# 
    node_inds_19 = [actor_ids.index(actor_id) for actor_id in df_19['track_id']]# 当前时刻所有actors
    node_positions_19 = torch.from_numpy(np.stack([df_19['rel_x'].values, df_19['rel_y'].values], axis=-1)).float()
    node_positions = torch.matmul(node_positions_19 - origin, rotate_mat).float()
    #获取特征 向量，是否交叉，转弯
    # (lane_vectors, is_intersections, turn_directions, traffic_controls, lane_actor_index,
    #  lane_actor_vectors) = get_lane_features(am, node_inds_19, node_positions_19, origin, rotate_mat, city, radius)

    y = x[:, 20:] # 轨迹gt 但是是处理过的 AV中心 而且差分
    seq_id = str(frame_to_get)


    # past + future 
    # target_obs_df =   obs_df[(obs_df['frame_idx'] <=  frame_to_get + 30) & (obs_df['frame_idx'] >= frame_to_get -19)  ]
    
    # print(target_obs_df.shape)
    

    # origin = torch.tensor([target_obs_df.iloc[frame_to_get]['rel_x'],target_obs_df.iloc[frame_to_get]['rel_y']],dtype=torch.double)
    # rotate_mat = torch.tensor([[1, 0],
    #                            [0, 1]])
    
    
    # print(origin)
    lanes_df = pd.read_csv(raw_path)

    num_rows = lanes_df.shape[0]
    num_columns = lanes_df.shape[1]
    target_lanes_df =   lanes_df[lanes_df['frame_idx'] ==   timestamps[19] ].iloc
    # print(target_lanes_df[0][3]) #columns x0y0->,3 ,4
    # print(type(target_lanes_df))
    lane_idx_start =  3
    x_pos = target_lanes_df[:, range(lane_idx_start, num_columns, 2)]
    y_pos = target_lanes_df[:, range(lane_idx_start + 1 , num_columns, 2)]
    x_pos_np = x_pos.to_numpy()
    y_pos_np = y_pos.to_numpy()
    #转 numpy
    # print('pos')
    # print((x_pos_np[:,:5]))
    # print((y_pos_np[:,:5]))
    # print(x_pos.iloc[1:])

    # print(x_pos.iloc[:,1:])
    # print(x_pos.iloc[:,:-1])
    #### boost lane pos
    
    # print(y_pos[:5])
    #############################################

    raw_one_x = np.hstack(x_pos_np)## 多行车道的x 变成一个
    raw_one_y = np.hstack(y_pos_np)
    raw_lane_pos  = np.vstack((raw_one_x, raw_one_y))## 组成2*n+1
    raw_lane_pos = torch.from_numpy(raw_lane_pos).float()# 2*(n+1)
    
    raw_lane_pos = raw_lane_pos.permute(1,0)#(n+1)*2
    inv_rotate_mat = rotate_mat.t()
    raw_lane_pos[:] = torch.matmul(raw_lane_pos[:],inv_rotate_mat)+ origin
    #############################原始数据可视化


####
    vector_x_pos_np = x_pos_np[:,1:] - x_pos_np[:,:-1]
    vector_y_pos_np = y_pos_np[:,1:] - y_pos_np[:,:-1]
    one_vx = np.hstack(vector_x_pos_np) #2* -->1*
    one_vy = np.hstack(vector_y_pos_np)
    lane_vector_np  = np.vstack((one_vx, one_vy))

    # print(' shapes :')
    # print(y_pos_np.shape)
    # print(vector_y_pos_np.shape)
    # print(one_vx.shape)
    # print(lane_vector_np.shape)
           
    # vector_x = x_pos.iloc[:,1:].values - x_pos.iloc[:,:-1].values
    # vector_y = y_pos.iloc[:,1:].values - y_pos.iloc[:,:-1].values # 前进方向
    # # print(vector_x)
    # lane_vector_np = np.concatenate((vector_x,vector_y), axis=1)
    lane_vectors = torch.from_numpy(lane_vector_np).float()
    lane_vectors = lane_vectors.permute(1,0) # L*2

    print(lane_vectors.shape)# unit lanes for av

    head_raw_one_x = np.hstack(x_pos_np[:,1:])## 多行车道的x 变成一个
    head_raw_one_y = np.hstack(y_pos_np[:,1:])
    lanes_position_np  = np.vstack((head_raw_one_x, head_raw_one_y))## 组成2*n+1

    # lanes_position_np = np.concatenate((x_pos.iloc[:,1:].values, y_pos.iloc[:,1:].values),axis =1 )

    print(lanes_position_np.shape)
    lanes_position = torch.from_numpy(lanes_position_np).float()
    lanes_position = lanes_position.permute(1,0)
    # get lanes map for all actors

    ones_tensor = torch.ones(lane_vectors.shape[0])
    is_intersections = ones_tensor == 0 # default false
    turn_directions = torch.zeros(lane_vectors.shape[0]) # none
    traffic_controls =  ones_tensor == 0 # default false

    lane_actor_index = torch.LongTensor(list(product(torch.arange(lane_vectors.size(0)), node_inds_19))).t().contiguous()
    # 都是相对自车的位置 和车道线 两者的差也是车道线和19阵时刻障碍物之间的距离
    lane_actor_vectors = \
        lanes_position.repeat_interleave(len(node_inds_19), dim=0) - node_positions.repeat(lane_vectors.size(0), 1)
    mask = torch.norm(lane_actor_vectors, p=2, dim=-1) < 50# 50m 以外的不关心
    lane_actor_index = lane_actor_index[:, mask]
    lane_actor_vectors = lane_actor_vectors[mask]
    # interaction 
    # 所有车道点的索引 和 agents 索引 的pair
    # lane_actor_index = torch.LongTensor(list(product(torch.arange(lane_vectors.size(0)), node_inds))).t().contiguous()#product() 可以接受不同类型序列作为输入


    return  {
        'x': x[:, : 20],  # [N, 20, 2] processed transformed differential history trajectory
        'positions': positions,  # [N, 50, 2]   comlete trajectory after transformed but before differential
        'edge_index': edge_index,  # [2, N x N - 1]
        'y': y,  # [N, 30, 2]
        'num_nodes': actor_num,
        'padding_mask': padding_mask,  # [N, 50]
        'bos_mask': bos_mask,  # [N, 20]
        'rotate_angles': rotate_angles,  # [N]
        'lane_vectors': lane_vectors,  # [L, 2]
        'is_intersections': is_intersections,  # [L]
        'turn_directions': turn_directions,  # [L]
        'traffic_controls': traffic_controls,  # [L]
        'lane_actor_index': lane_actor_index,  # [2, E_{A-L}]
        'lane_actor_vectors': lane_actor_vectors,  # [E_{A-L}, 2]
        'seq_id': int(seq_id),
        'av_index': av_index,
        'agent_index': agent_index,
        'city': 'HeFei',
        'origin': origin.unsqueeze(0),
        'theta': theta,
        'raw_lane_pos': raw_lane_pos, #n*2
        'complete_samples': complete_samples
    }

In [191]:
kwargs = get_lanes(raw_path='/home/alon/Learning/HiVT/mini_data/lanes_dlp_training.csv',
          raw_path_obs = '/home/alon/Learning/HiVT/mini_data/obs_dlp_training2.csv',
          frame_to_get = 44)
_data = TemporalData(**kwargs)#封装成自定义数据类型
print(_data)
torch.save(_data, os.path.join('/home/alon/Learning/HiVT/mini_data/', str(kwargs['seq_id']) + '.pt'))# 根据目录存为pt文件

from to  25 74
actors ids : [-1, 316, 292, 491, 583, 595, 303]
origin: tensor([12456.2666,  2710.2927])
av theta in boost at 19 stamps  tensor(-0.1900)
full node_historical_steps stamps: 0
full _steps stamps: 0
full node_historical_steps stamps: 2
full _steps stamps: 2
full node_historical_steps stamps: 3
full _steps stamps: 3
full node_historical_steps stamps: 4
torch.Size([372, 2])
(2, 372)
TemporalData(agent_index=2, av_index=0, bos_mask=[7, 20], city="HeFei", complete_samples=[3], edge_index=[2, 42], is_intersections=[372], lane_actor_index=[2, 653], lane_actor_vectors=[653, 2], lane_vectors=[372, 2], origin=[1, 2], padding_mask=[7, 50], positions=[7, 50, 2], raw_lane_pos=[375, 2], rotate_angles=[7], seq_id=44, theta=-0.19004382193088531, traffic_controls=[372], turn_directions=[372], x=[7, 20, 2], y=[7, 30, 2])
